In [ ]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential, Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# Small dataset ~ 300 images for 2 classes

In [ ]:
img_width, img_height = 224, 224
train_data_dir = "image_small/training_set"
validation_data_dir = "image_small/test_set"

In [ ]:
model = applications.VGG16(weights='imagenet',
                           include_top=False, #tắt chức năng classification để tự thêm vào sau
                           input_shape=(img_width, img_height,3))

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

#Dataset nhỏ nhưng giống dataset gốc

In [ ]:
#Đóng băng VGG16 layer.
for layer in model.layers:
  layer.trainable=False

In [ ]:
#thêm các layers tùy biến (ANN)
x = model.output #lấy kết quả của model
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='sigmoid')(x)

In [ ]:
#creating the final model
model_final = Model(inputs = model.input, outputs=predictions)

In [ ]:
#compile the model
model_final.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
                    metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True,
                                   fill_mode = "nearest",
                                   zoom_range = 0.3,
                                   width_shift_range = 0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height, 
                                                                   img_width),
                                                    batch_size = 32, 
                                                    class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                    target_size = (img_height, 
                                                                   img_width),
                                                    class_mode = "categorical")


Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [ ]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("owl_and_eagle_small_vgg16.h5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq=1)
early = EarlyStopping(monitor='val_loss', 
                      min_delta=0.001, 
                      patience=10, 
                      verbose=1, 
                      mode='auto')


In [ ]:
import datetime

In [ ]:
t0 = datetime.datetime.now()
print(t0)

2021-11-08 14:53:32.909524


In [ ]:
#train the model
history = model_final.fit(train_generator,
                          steps_per_epoch = len(train_generator), #100 mẫu
                          epochs=100,
                          validation_data = validation_generator,
                          validation_steps=len(validation_generator),
                          verbose=1,
                          callbacks = [checkpoint, early])

Epoch 1/100
7/7 [==============================] - 169s 22s/step - loss: 0.8467 - accuracy: 0.4850 - val_loss: 0.7408 - val_accuracy: 0.5000
Epoch 2/100
7/7 [==============================] - 5s 737ms/step - loss: 0.8235 - accuracy: 0.5150 - val_loss: 0.7724 - val_accuracy: 0.5100
Epoch 3/100
7/7 [==============================] - 5s 737ms/step - loss: 0.8157 - accuracy: 0.4350 - val_loss: 0.6828 - val_accuracy: 0.5100
Epoch 4/100
7/7 [==============================] - 5s 733ms/step - loss: 0.7154 - accuracy: 0.5550 - val_loss: 0.6685 - val_accuracy: 0.5800
Epoch 5/100
7/7 [==============================] - 5s 809ms/step - loss: 0.6497 - accuracy: 0.5900 - val_loss: 0.6636 - val_accuracy: 0.5900
Epoch 6/100
7/7 [==============================] - 5s 728ms/step - loss: 0.6639 - accuracy: 0.5950 - val_loss: 0.6602 - val_accuracy: 0.5700
Epoch 7/100
7/7 [==============================] - 5s 815ms/step - loss: 0.6302 - accuracy: 0.6550 - val_loss: 0.6430 - val_accuracy: 0.6300
Epoch 8/100
7

In [ ]:
t1 = datetime.datetime.now()
print(t1)

2021-11-08 15:00:42.885001


In [ ]:
print(t1 - t0)

0:07:09.975477


In [ ]:
import pandas as pd

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.tail()

,loss,accuracy,val_loss,val_accuracy
41,0.379996,0.870,0.566499,0.66
42,0.400384,0.825,0.519795,0.72
43,0.335304,0.850,0.560641,0.69
44,0.322970,0.880,0.531311,0.71
45,0.327456,0.885,0.531365,0.70


In [ ]:
#save model
from tensorflow.keras.models import load_model
model_final.save('owl_and_eagle_small_vgg16_final.h5')
print('save!!!')

save!!!
